In [ ]:
pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 42.8 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyspark: filename=pyspark-3.5.4-py2.py3-none-any.whl size=317849827 sha256=dd896da64829c046f76fae6896ec683734ea0bc077186ad38bb116ca5b7ef967
  Stored in directory: /home/codespace/.cache/pip/wheels/13/92/64/da92a3521323cc629fdf25dd56eb26938e08014c1b57ad3759
Successfully built pyspark

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("my_app").getOrCreate()



25/02/24 19:55:21 WARN Utils: Your hostname, codespaces-7aea8b resolves to a loopback address: 127.0.0.1; using 10.0.4.143 instead (on interface eth0)
25/02/24 19:55:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/24 19:55:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.range(500).toDF('number')
df.head()

Row(number=0)

In [14]:
df = df.select((df['number'] + 10).alias('number')).withColumnRenamed('number','new_number')
df.show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `number` cannot be resolved. Did you mean one of the following? [`(number + 10)`].

In [11]:
spark.read.format('json').load('/workspaces/retail_d/data/flight-data/json/2015-summary.json').schema


StructType([StructField('DEST_COUNTRY_NAME', StringType(), True), StructField('ORIGIN_COUNTRY_NAME', StringType(), True), StructField('count', LongType(), True)])

In [ ]:
# to read the json format file from local by giving relative path of file
df = spark.read.format('json').load('/workspaces/retail_d/data/flight-data/json/2015-summary.json')

In [ ]:
#to see the schema in tree structure like name,datatype,and boolean value whether that column can be null or not.
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [ ]:
#to see the columns
df.columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

In [ ]:
#this first() is used to retrieve the very first row of the dsata frame .you see it returnS as Row object in spark with data in it.
df.first()

Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)

In [ ]:
df = spark.read.format("json").load("/workspaces/retail_d/data/flight-data/json/2015-summary.json")
df.createOrReplaceTempView("dfTable")#this creates a temporary sql table

In [25]:
df = spark.sql('select * from dfTable')
df.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [24]:
spark.sql('SELECT *, (DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry FROM dfTable LIMIT 2').show()

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+



In [27]:
df.selectExpr("avg(count)", "count(distinct(DEST_COUNTRY_NAME))").show()

+-----------+---------------------------------+
| avg(count)|count(DISTINCT DEST_COUNTRY_NAME)|
+-----------+---------------------------------+
|1770.765625|                              132|
+-----------+---------------------------------+



In [40]:
#spark.sql('SELECT *, 1 as One FROM dfTable LIMIT 2').show()#sql literal here we are just passing another column one with value 1 just literal
#same way we have in pyspark 
from pyspark.sql.functions import lit,expr # here lit is to give literal value
#df.select(expr('*'),lit(1).alias(one)).show(2)
#df.selectExpr('*', '1 as one').show(2)# all the above i am doing are same but in diferent formats
# we can achieve similar functionality using withColumn
df.withColumn('one',lit(1)).show(2) 
# with column comes in handy when we want to add new column in the dataframe based on expressesion
df.withColumn('withincountry',expr('DEST_COUNTRY_NAME == ORIGIN_COUNTRY_NAME')).show(2)


+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|one|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withincountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [43]:
#renaming columns
df.withColumnRenamed('DEST_COUNTRY_NAME','dest').columns

['dest', 'ORIGIN_COUNTRY_NAME', 'count']

In [ ]:
# to create dataframe using row value and defined schema
from pyspark.sql import Row 
from pyspark.sql.types import StructType,StructField,StringType,LongType
mymanualschema = StructType([
  StructField("some", StringType(), True),
  StructField("col", StringType(), True),
  StructField("names", LongType(), False)
])
myrow =Row('Hello','None',1)
df = spark.createDataFrame([myrow],mymanualschema)
df.show()


+-----+----+-----+
| some| col|names|
+-----+----+-----+
|Hello|None|    1|
+-----+----+-----+



In [14]:
df.select('some').show()

+-----+
| some|
+-----+
|Hello|
+-----+



In [ ]:
df.selectExpr()# can helps inline column renaming,transformations like we do in sql

In [ ]:
seed = 5
withReplacement = False
fraction = 0.5
df.sample(withReplacement, fraction, seed).count()


+-------------+
|(number + 10)|
+-------------+
|           10|
|           11|
|           12|
|           13|
|           14|
|           15|
|           16|
|           17|
|           18|
|           19|
|           20|
|           21|
|           22|
|           23|
|           24|
|           25|
|           26|
|           27|
|           28|
|           29|
+-------------+
only showing top 20 rows



In [2]:
new_df = spark.read.format('csv')\
        .option('header','true')\
        .option('inferSchema','true')\
        .load('/workspaces/retail_d/data/retail-data/by-day/2010-12-01.csv')

In [5]:
new_df.rdd.getNumPartitions()
new_df.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|     7.65|   17850.0|United Kingdom|
|   536365|    21730|GLASS S

In [8]:
new_df.schema

StructType([StructField('InvoiceNo', StringType(), True), StructField('StockCode', StringType(), True), StructField('Description', StringType(), True), StructField('Quantity', IntegerType(), True), StructField('InvoiceDate', TimestampType(), True), StructField('UnitPrice', DoubleType(), True), StructField('CustomerID', DoubleType(), True), StructField('Country', StringType(), True)])

In [ ]:
from pyspark.sql.functions import lit,col
new_df.select(lit(5),lit('five'),lit(5.0))

new_df.where(col('InvoiceNo')=='536368').select('InvoiceNo','Description').show(5,False)# this False arguement says if the column content is two long dont truncate just display content as it is.

+---------+------------------------------+
|InvoiceNo|Description                   |
+---------+------------------------------+
|536368   |JAM MAKING SET WITH JARS      |
|536368   |RED COAT RACK PARIS FASHION   |
|536368   |YELLOW COAT RACK PARIS FASHION|
|536368   |BLUE COAT RACK PARIS FASHION  |
+---------+------------------------------+

